# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention/results/2024_01_30_rce_2nd_cohort/proc/rce_pilot_2_full_spectral_and_sleap_poses.pkl")

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [9]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,subject_locations,agent_locations,file_path,tracked_subject,in_video_subjects,notes,sleap_filename,sleap_fileroot,start_stop_frame,start_stop_timestamps
0,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[[46.68890098287189, 11.495085446056166], [43...","[[[34.36404036584844, 35.615553316175784], [34...",/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",1.1_1.2,NaN,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"(1, 32300)","(2444, 32361438)"
1,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[[34.36404036584844, 35.615553316175784], [34...","[[[46.68890098287189, 11.495085446056166], [43...",/scratch/back_up/reward_competition_extention/...,"[1.1, 1.2]",1.1_1.2,NaN,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"(1, 32300)","(2444, 32361438)"
2,2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.1,...,"[[[46.71964110607613, 36.493379374373234], [44...","[[[50.63276479731981, 10.511614189582007], [51...",/scratch/back_up/reward_competition_extention/...,"[1.1, 1.4]",1.1_1.4,NaN,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"(1, 33000)","(1384, 33062688)"
3,2,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"[[916, 1117], [3312, 3513], [5608, 5808], [740...","[[49, 67], [70, 79], [360, 366], [460, 469], [...","[[33601, 33798], [34108, 34165], [34166, 34179...",20230613_105657_standard_comp_to_training_D2_s...,/scratch/back_up/reward_competition_extention/...,20230613_105657_standard_comp_to_training_D2_s...,1.4,...,"[[[50.63276479731981, 10.511614189582007], [51...","[[[46.71964110607613, 36.493379374373234], [44...",/scratch/back_up/reward_competition_extention/...,"[1.1, 1.4]",1.1_1.4,NaN,20230613_105657_standard_comp_to_training_D2_s...,20230613_105657_standard_comp_to_training_D2_s...,"(1, 33000)","(1384, 33062688)"
4,2,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,"[[87, 287], [2483, 2683], [4778, 4979], [6575,...","[[338, 339], [341, 385], [385, 482], [696, 703...","[[31729, 31729], [31731, 31732], [31733, 31735...",20230616_111904_standard_comp_to_training_D4_s...,/scratch/back_up/reward_competition_extention/...,20230616_111904_standard_comp_to_training_D4_s...,1.2,...,"[[[51.59712734869665, 31.240714372660907], [56...","[[[56.75056981673038, 24.705250867385512], [55...",/scratch/back_up/reward_competition_extention/...,"[1.2, 1.4]",1.2_1.4,NaN,20230616_111904_standard_comp_to_training_D4_s...,20230616_111904_standard_comp_to_training_D4_s...,"(1, 31100)","(1383, 31159889)"


## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [10]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [51]:
FULL_LFP_TRACES_PKL = "{}_04_lfp_spectral_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [12]:
ALL_PHY_DIR

['/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/20230617_115521_standard_comp_to_omi

In [13]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [14]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,3,71,22,243,8.729426,1,0,20.0,9.474292,mua,33069,0,5
1,5,4,0,14,9.769127,2,0,40.0,1.879731,mua,6561,0,7
2,6,2,0,11,13.532221,2,0,40.0,11.421351,good,39865,0,8
3,7,58,30,99,9.302568,2,0,40.0,20.851808,good,72781,0,9
4,8,29,12,94,7.241092,3,0,60.0,4.613521,mua,16103,0,10


- Combining all the unit info dataframes and adding the recording name

In [15]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [16]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,0,12,9,45,10.498872,0,0.0,0.0,4.379580,mua,14970,0,1.0
1,20230620_114347_standard_comp_to_omission_D4_s...,11,0,0,0,10.311028,5,0.0,100.0,0.230828,mua,789,0,12.0
2,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
3,20230620_114347_standard_comp_to_omission_D4_s...,21,148,57,355,9.774931,11,0.0,220.0,10.004284,mua,34196,0,23.0
4,20230620_114347_standard_comp_to_omission_D4_s...,22,105,26,351,8.435524,12,0.0,240.0,13.887395,mua,47469,0,24.0


- Filtering for the good units

In [17]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [18]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
1,20230620_114347_standard_comp_to_omission_D4_s...,28,2,1,7,8.708598,14,0.0,280.0,1.436163,good,4909,0,30.0
2,20230620_114347_standard_comp_to_omission_D4_s...,41,2,1,3,10.409801,19,0.0,380.0,1.583319,good,5412,0,43.0
3,20230620_114347_standard_comp_to_omission_D4_s...,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0
4,20230620_114347_standard_comp_to_omission_D4_s...,43,1,0,4,7.047093,26,0.0,520.0,1.489701,good,5092,0,45.0


In [19]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [20]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_clusters.npy'


In [21]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 42,  42, 207, ...,  22, 229,  68], dtype=int32)

In [22]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(903326,)

- The times that all the spikes happened

In [23]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_times.npy'


In [24]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[     120],
       [     262],
       [     439],
       ...,
       [68362487],
       [68362524],
       [68362610]])

In [25]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(903326, 1)

### Combining everything into a dataframe

In [26]:
recording_to_spike_df = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [27]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,2,0,0,0,24.692951,1,0.0,20.0,1.070175,good,3658,0,3.0
1,3,235,106,472,10.347874,1,0.0,20.0,14.493880,mua,49542,0,4.0
2,4,64,25,149,11.953332,4,0.0,80.0,27.163371,mua,92848,0,5.0
3,10,69,29,194,10.297276,18,0.0,360.0,6.455279,mua,22065,0,12.0
4,11,619,261,1295,9.638424,19,0.0,380.0,18.195901,mua,62196,0,13.0


In [28]:
spike_times

array([[     160],
       [     483],
       [     503],
       ...,
       [68361973],
       [68362204],
       [68362381]])

In [29]:
spike_clusters

array([ 4,  4, 11, ...,  4,  4, 87], dtype=int32)

- Combining the spike time df for all recordings

In [30]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [31]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [32]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,42,120,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
1,42,262,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,142.0,0.00710
2,207,439,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
3,207,632,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,193.0,0.00965
4,262,723,262,0,0,0,20.012720,19,0.0,380.0,2.037368,good,6964,0,39.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN


In [33]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
2017369,64,68356117,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,392.0,0.0196
2017370,89,68356258,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,3360.0,0.1680
2017371,64,68357205,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1088.0,0.0544
2017372,89,68357578,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,1320.0,0.0660
2017373,64,68358945,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1740.0,0.0870


## Grouping all the neurons by recording

In [34]:
# Grouping all spike times by neuron and recording

grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: sorted(list(x))).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: list(x)}).reset_index()

In [35]:
grouped_df

,recording_name,spike_clusters,spike_times
0,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ..."
1,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2..."
2,20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120..."
3,20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011..."
4,20230619_115321_standard_comp_to_omission_D3_s...,"[11, 44, 132, 134, 186, 187, 227, 250, 252, 25...","[[17747, 20655, 24011, 34755, 67305, 110220, 1..."
5,20230620_114347_standard_comp_to_omission_D4_s...,"[2, 22, 64, 89, 98, 100, 119, 121, 122, 126]","[[264055, 325873, 327931, 330996, 334463, 3590..."
6,20230620_114347_standard_comp_to_omission_D4_s...,"[14, 28, 41, 42, 43, 45, 48, 53, 58, 68, 124, ...","[[5884, 7172, 11405, 19311, 19371, 23729, 2627..."
7,20230621_111240_standard_comp_to_omission_D5_s...,"[6, 7, 13, 15, 16, 17, 22, 24, 26, 36, 37, 46,...","[[1891, 5872, 8366, 16312, 17233, 24302, 24337..."


In [36]:
LFP_SPECTRAL_DF = pd.merge(LFP_SPECTRAL_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

## Calculating the firing rates

In [37]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'subject', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_all_windows',
     

In [38]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject,...,tracked_subject,in_video_subjects,notes,sleap_filename,sleap_fileroot,start_stop_frame,start_stop_timestamps,recording_name,spike_clusters,spike_times
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"[1.1, 1.4]",1.1_1.4,NaN,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120..."
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"[1.1, 1.4]",1.1_1.4,NaN,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011..."


In [39]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [40]:
LFP_SPECTRAL_DF["neuron_average_timestamps"] = LFP_SPECTRAL_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [41]:
LFP_SPECTRAL_DF["neuron_average_fr"] = LFP_SPECTRAL_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [42]:
LFP_SPECTRAL_DF.iloc[:5,:10]

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,subject
0,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1
1,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4


In [43]:
LFP_SPECTRAL_DF.iloc[:5,10:20]

,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace
0,"[1.1, 1.4]",835680,66439621,"[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[[33057437, 33069437], [33070037, 33075039], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.3154, -0.2812, -0.2698, -0.3381, -0.2261, ...","[0.003227, -0.06616, -0.0742, 0.0565, 0.1743, ..."
1,"[1.1, 1.4]",835680,69429292,"[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[[33057437, 33069437], [33070037, 33075039], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0823, 0.0658, -0.1316, -0.7896, -1.333, -1....","[-0.4082, -0.651, -0.852, -0.8345, -0.8223, -0..."


In [44]:
LFP_SPECTRAL_DF.iloc[:5,20:30]

,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps,power_calculation_frequencies
0,"[0.3372, 0.4973, 0.524, 0.3906, 0.4954, 0.56, ...","[0.1832, 0.4846, 0.766, 0.8545, 0.961, 0.7627,...","[0.9727, 1.524, 2.242, 2.426, 2.252, 1.847, 1....","[[0.015305, 0.0223, 0.03632, 0.02211, 0.02168,...","[[0.00202, 0.00401, 0.012985, 0.011696, 0.0151...","[[0.006813, 0.00756, 0.01591, 0.01199, 0.01014...","[[0.01706, 0.02808, 0.02686, 0.01572, 0.02292,...","[[0.01005, 0.01069, 0.01654, 0.0264, 0.02428, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,"[-0.846, -1.169, -1.349, -1.064, -0.8716, -0.9...","[-0.1432, -0.525, -1.298, -1.044, -0.3499, -1....","[-0.755, -1.41, -2.273, -2.297, -2.133, -2.557...","[[0.02173, 0.01424, 0.01416, 0.00531, 0.03857,...","[[0.004864, 0.004448, 0.0083, 0.009125, 0.0586...","[[0.002157, 0.002275, 0.00783, 0.009575, 0.071...","[[0.0183, 0.00981, 0.02113, 0.01231, 0.0809, 0...","[[0.005535, 0.00698, 0.03174, 0.03128, 0.1538,...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."


In [45]:
LFP_SPECTRAL_DF.iloc[:5, 30:]

,BLA_LH_coherence_all_frequencies_all_windows,BLA_MD_coherence_all_frequencies_all_windows,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,mPFC_vHPC_coherence_all_frequencies_all_windows,...,notes,sleap_filename,sleap_fileroot,start_stop_frame,start_stop_timestamps,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[[0.539, 0.8413, 0.714, 0.6323, 0.6743, 0.8394...","[[0.0107, 0.5386, 0.4678, 0.5063, 0.715, 0.814...","[[0.818, 0.9517, 0.8213, 0.7104, 0.537, 0.1165...","[[0.8555, 0.9033, 0.846, 0.6855, 0.4634, 0.003...","[[0.5137, 0.8047, 0.8423, 0.8394, 0.783, 0.743...","[[0.1504, 0.6855, 0.815, 0.9004, 0.836, 0.3022...","[[0.86, 0.8716, 0.9053, 0.695, 0.647, 0.00902,...","[[0.06226, 0.4448, 0.678, 0.85, 0.6284, 0.2715...","[[0.1669, 0.482, 0.66, 0.351, 0.2292, 0.0636, ...","[[0.4548, 0.7715, 0.8076, 0.614, 0.6577, 0.531...",...,NaN,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[[0.811, 0.2367, 0.7607, 0.7485, 0.913, 0.8623...","[[0.973, 0.3767, 0.662, 0.607, 0.9136, 0.8486,...","[[0.978, 0.9204, 0.6987, 0.4927, 0.811, 0.7563...","[[0.00341, 0.292, 0.4998, 0.866, 0.9854, 0.994...","[[0.6675, 0.9556, 0.927, 0.931, 0.997, 0.9985,...","[[0.91, 0.4858, 0.5474, 0.7476, 0.9727, 0.974,...","[[0.1449, 0.68, 0.7188, 0.783, 0.918, 0.8477, ...","[[0.9062, 0.6196, 0.634, 0.7397, 0.9766, 0.971...","[[0.0485, 0.7505, 0.481, 0.582, 0.9175, 0.8296...","[[0.00795, 0.3496, 0.2861, 0.59, 0.8247, 0.751...",...,NaN,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"(1, 32316)","(1384, 32376683)",20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."


In [46]:
LFP_SPECTRAL_DF["neuron_average_timestamps"].iloc[0].shape

(32801,)

In [47]:
LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0].shape

(21, 32801)

In [48]:
np.max(LFP_SPECTRAL_DF["neuron_average_fr"].iloc[0])

10.0

In [52]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [50]:
raise ValueError("")

ValueError: 